# Surveying recent events and planning your followup

We've now covered all the endpoints provided by `voeventdb`. In this notebook, we'll look at how to request particular list-orderings, and demonstrate how you might use `voeventdb` to plan a night's follow-up observing.

In [ ]:
from __future__ import print_function
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys
from voeventdb.remote.helpers import Synopsis
from datetime import datetime, timedelta
import pytz

## List ordering
When using the `voeventdb` list-query endpoints, we can control ordering using the `order` parameter. So for example, if we simply request the first 50 IVORNs with no modifiers, we get the first entries inserted into the database:

In [ ]:
apiv1.list_ivorn(n_max=5)

In this case, whatever happened to arrive just after the database was switched on, before the previous year's events were loaded from an archive.
But, we can change the ordering, choosing from one the 
[order-values](http://voeventdbremote.readthedocs.org/en/latest/reference/index.html#voeventdb.remote.apiv1.OrderValues). For example, to retrieve the most recent 'observation' VOEvents, we'll request the list in 'author-datetime descending' order:

In [ ]:
apiv1.list_ivorn(filters={FilterKeys.role:'observation'},
                 order=apiv1.OrderValues.author_datetime_desc,
                 n_max=5, 
                )

## Retrieving recent GRB events
With this last feature, we can start using `voeventdb` for applications such as reviewing the most recent alerts, and perhaps even planning our follow-up. 
Suppose we want to retrieve the 10 most-recent GRB-event alerts from the *Swift* BAT:

In [ ]:
filters = { FilterKeys.ivorn_contains: 'BAT_GRB',
            FilterKeys.role: 'observation'}
now = datetime.utcnow()
recent_swift_grb_ivorns = apiv1.list_ivorn(filters,
                                     order=apiv1.OrderValues.author_datetime_desc,
                                     n_max=10,
                                    )
recent_swift_grbs = [Synopsis(apiv1.packet_synopsis(i)) for i in recent_swift_grb_ivorns]

Let's view a summary table, displaying just the dates and co-ords:

In [ ]:
print("Recent GRBs as of {}:".format(now))
print()
for grb in recent_swift_grbs:
    print(grb.author_datetime, grb.coords[0].ra.deg, grb.coords[0].dec.deg)

If we wanted to be a little fancy, we could use a [pandas](http://pandas.pydata.org/) dataframe to draw the table for us, applying some formatting tricks along the way:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(index = [g.author_datetime for g in recent_swift_grbs],
                  data = {'ra': [g.coords[0].ra.deg for g in recent_swift_grbs],
                          'dec': [g.coords[0].dec.deg for g in recent_swift_grbs],
    })
#Re-order the columns
df = df[['ra','dec']]
#Set the float-display formatting:
pd.options.display.float_format = '{:+07.2f}'.format
print("Recent GRBs as of {}:".format(now))
df

## Wrapping up

That about covers all the major features of ``voeventdb``, as demonstrated using the ``voeventdb.remote`` client-library. 
Comments, questions, bug-reports and other contributions are all welcomed - you can leave a note on the 
[issue tracker](https://github.com/timstaley/voeventdb.remote/issues) or find more contact details at http://4pisky.org/voevents/.

## And finally...
We've covered how to get data out of `voeventdb` - but not what to do with the data when you've got it. The examples notebook demonstrates a few basic ideas - see next!